In [1]:
#Before running this notebook, make sure to create a backup of metadata table.

# import datetime
# now = datetime.datetime.now()
# bkp_path = '/mnt/customer360-blob-output/C360/'+now.strftime("%Y_%m_%d_%H_%M_%S")+'/metadata_table_bkp/'
#dbutils.fs.cp('/mnt/customer360-blob-output/C360/metadata_table/', bkp_path)

In [2]:
metadata_table_path = '/mnt/customer360-blob-output/C360/metadata_table/'
#metadata_table_path = '/mnt/customer360-blob-data/C360/data_dependency_check/metadata_table/'
dataset_name_for_reset = 'test'
dataset_path_for_reset = '/mnt/customer360-blob-output/C360/REVENUE/l4_features/test/'
reset_date_for_dataset = '1970-01-01'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [4]:
def _get_spark():
  spark = SparkSession.builder.appName("project_customer_360_data_flow_control").getOrCreate()
  spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
  spark.conf.set("spark.sql.parquet.binaryAsString", "true")
  spark.conf.set("spark.sql.shuffle.partitions", 200)
  spark.conf.set("spark.sql.files.maxPartitionBytes", 1024*1024*256)
  return spark
spark = _get_spark()
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

In [5]:
mtdt_tbl = spark.read.parquet(metadata_table_path)

In [6]:
metadata_table_update_df = spark.range(1)

metadata_table_update_df = (
            metadata_table_update_df.withColumn("table_name", F.lit(dataset_name_for_reset))
                .withColumn("table_path", F.lit(dataset_path_for_reset))
                .withColumn("write_mode", F.lit("overwrite"))
                .withColumn("target_max_data_load_date", F.to_date(F.lit(reset_date_for_dataset), "yyyy-MM-dd"))
                .withColumn("updated_on", F.current_date())
                .drop("id")
                    )

metadata_table_update_df.write.partitionBy("table_name").format("parquet").mode("overwrite").save(metadata_table_path)